In [1]:
2. Apply a few ML algorithms.
 - We'll use the taxi data first.
      - The taxi pricing model is very clear and there is no surging
      - There is more taxi data, and the model is known to be consistent since Jan 2016 (see prev. notebook)
 - Linear model with the extra categorical variables included. 

SyntaxError: invalid syntax (<ipython-input-1-74c0d038b9d1>, line 1)

### ACTUALLY DO MACHINE LAERN
https://ramhiser.com/post/2018-04-16-building-scikit-learn-pipeline-with-pandas-dataframe/


In [ ]:
class TripPreprocessor(BaseEstimator, TransformerMixin):
    def __init__(self, max_fare = 200.0, max_miles = 100.0):
        self.max_fare = max_fare
        self.max_miles = max_miles
    def fit(self, X, y=None):
        return self  # nothing else to do
    def transform(self, X, y=None):

class TypeSelector(BaseEstimator, TransformerMixin):
    def __init__(self, dtype):
        self.dtype = dtype

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        assert isinstance(X, pd.DataFrame)
        return X.select_dtypes(include=[self.dtype])


preprocess_pipeline = make_pipeline(
    FeatureUnion(transformer_list=[
        ("numeric_features", make_pipeline(
            TypeSelector(np.number),
            StandardScaler()
        )),
        ("categorical_features", make_pipeline(
            TypeSelector("category"),
            OneHotEncoder()
        )),
        ("boolean_features", make_pipeline(
            TypeSelector("bool")
        ))
    ])
)

preprocess_pipeline.fit(taxi)

taxi.dtypes

X = taxi.drop(["Final_Fare","Trip_Start_Timestamp"], axis = 1)
y = taxi["Final_Fare"]

lin_reg = LinearRegression(preprocess_pipeline)
lin_reg.fit(X,y)

lin_reg

def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())

lin_scores = cross_val_score(lin_reg, X, y,
                             scoring="neg_mean_squared_error", cv=10)
lin_rmse_scores = np.sqrt(-lin_scores)
display_scores(lin_rmse_scores)

taxi.info()